# Жесткие системы ОДУ 

## Задача  
Модель дифференциации растительной ткани

Данный пример - типичный случай биохимической модели
«умеренной» размерности (современные модели, например, фотосинтеза
включают сотни уравнений подобного типа). Хотя данная модель является
умеренно жесткой, тем не менее ее лучше решать с помощью методов,
предназначенных для решения ЖС ОДУ
представленного в виде системы двуъ ОДУ первого порядка:

<img src='pictures/system.png'>

##### Решить задачу методом второго порядка и выше, а также сравнить с явным методом.
1) ##### Неявный метод типа Рунге-Кутты 
2) ##### Метод Розенброка (Розенброка-Ванера)
3) ##### Неявный методы Адамса/ФДН (методы Гира) в представлении Нордсика

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import scipy.optimize as op
import matplotlib.cm as cm

Начальные данные и правая часть: 

In [ ]:
y0 = np.zeros(8)
y0[0] = 1
y0[7] = 0.0057

Tk = 421.8122

def f(t, y):
    y1 = - 1.71*y[0] + 0.43*y[1] +  8.23*y[2] + 0.0007
    y2 = + 1.71*y[0] - 8.75*y[1]
    y3 = -10.03*y[2] + 0.43*y[3] + 0.035*y[4]
    y4 = + 8.32*y[1] + 1.71*y[2] -  1.12*y[3] 
    y5 = -1.745*y[4] + 0.43*y[5] +  0.43*y[6]
    y6 = -280*y[5]*y[7] + 0.69*y[3] + 1.71*y[4] - 0.43*y[5] + 0.69*y[6]
    y7 = +280*y[5]*y[7] - 1.87*y[6]
    y8 = -y7

    return np.array([y1, y2, y3, y4, y5, y6, y7, y8])

#### Для сравнения: явный метод Рунге-Кутты второго порядка аппроксимации

In [ ]:
def rungekutta2_iter(y, t, tau, idx):
    k1 = tau * f(t[idx], y[idx])
    k2 = tau * f(t[idx] + tau/2, y[idx] + k1/2)
    k3 = tau * f(t[idx] + tau, y[idx] - k1 + 2 * k2)
    return y[idx] + (k1 + 4 * k2 + k3) / 6

def rungekutta2(y0, tau, T):
    t = np.arange(0, T, tau)
    n = int(T/tau)
    y = np.zeros((n, len(y0)))
    
    y[0] = y0

    for idx in range(n-1):
        y[idx+1] = rungekutta2_iter(y, t, tau, idx)

    return t, y

#### Построение графиков

In [ ]:
def graph(t, y, title, filename):
    
    plt.figure(figsize=[10, 4])

    colors = cm.rainbow(np.linspace(0, 1, len(y[0])))
    
    for ind in range(len(y[0])):
        plt.scatter(t[:len(y)], y[:,ind], color=colors[ind], label=f'y{ind+1}', s=0.1)
    
    plt.title(title)
    plt.grid()
    plt.xlabel('y')
    plt.ylabel('t')

    plt.legend(loc="upper right")
    
    plt.savefig(filename)

In [ ]:
t_rk2, y_rk2 = rungekutta2(y0, 0.015, 50)
graph(t_rk2, y_rk2, 'Явный метод Рунге-Кутты 2-го порядка, tau=0.015', 'pictures/rk2.1.png')

<img src='pictures/rk2.1.png'>

In [ ]:
t_rk2, y_rk2 = rungekutta2(y0, 0.01, 50)
graph(t_rk2, y_rk2, 'Явный метод Рунге-Кутты 2-го порядка, tau=0.01', 'pictures/rk2.2.png')

<img src='pictures/rk2.2.png'>

## 1. Неявный метод Рунге-Кутты

$k_{i} = f(t_{n} + c_{i}\tau, u^n + \tau\sum_{j=1}^{r} {a_{ij}k_{j}})$

$u^{n+1} = u^n + \tau(b_{1}k_{1} + ... + b_{r}k_{r})$

Будем использовать метод прямоугольников 2-го порядка.

|     |     |
|-----|-----|
| 1/2 | 1/2 |
|     | 1   |

### $\frac{u^{n+1} - u^n}{\tau} = f(u^{n+\frac{1}{2}}, t_{n+\frac{1}{2}})$

#### $t_{n+\frac{1}{2}} = \frac{t_{n+1} + t_{n}}{2}$

#### $u^{n+\frac{1}{2}} = \frac{u^{n+1} + u^{n}}{2}$

Подставив выражения:

#### $u^{n+1} = u^n + \tau f(\frac{u^{n+1} + u^{n}}{2}, \frac{t_{n+1} + t_{n}}{2})$

Или:

#### $u^{n+1} = u^n + \tau f(\frac{1}{2}u^{n+1} + \frac{1}{2}u^{n}, t_{n} + \frac{1}{2}\tau)$

In [ ]:
class System:
    def __init__(self, u_prev, t_prev, tau):
        self.u_prev = u_prev
        self.t_prev = t_prev
        self.tau = tau

    def __call__(self, u):
        return u - self.u_prev - self.tau * f(self.t_prev + self.tau/2, self.u_prev/2 + u/2)

def rect(y0, tau, T):
    t = np.arange(0, T, tau)
    n = int(T/tau)
    y = np.zeros((n, len(y0)))
    
    y[0] = y0

    for idx in range(n-1):
        system = System(y[idx], t[idx], tau)
        y[idx+1] = op.fsolve(system, y[idx])

    return t, y

#### Построение графиков

In [ ]:
t_rect, y_rect = rect(y0, 0.01, 50)
graph(t_rect, y_rect, 'Метод прямоугольников 2-го порядка', 'pictures/rect.png')

<img src='pictures/rect.png'>

## 2. Метод Розенброка

$k_{i} = \tau f(u_{n} + \sum_{j=1}^{i-1}{\beta_{i,j}k_{j}}) + \tau B \sum_{j=1}^{i}{\mu_{i,j}k_{j}}$

$u^{n+1} = u^n + \sum_{k=1}^{S}{\gamma_{k}k_{k}}$

Схема Розенброка с комплексными коэффициентами
второго порядка (CROS)
(Е.Н. Аристова, А. И. Лобанов ПРАКТИЧЕСКИЕ ЗАНЯТИЯ ПО ВЫЧИСЛИТЕЛЬНОЙ МАТЕМАТИКЕ В МФТИ, ЧАСТЬ II)

$(E-\frac{1+i}{2}\tau J)\omega = f(t^{n} + \frac{\tau}{2}, y^{n})$

$y^{n+1} = y^{n} + \tau Re(\omega)$

In [ ]:
def get_extended_matrix(matrix, f):
  return np.hstack((matrix,np.array([f]).T))

def gauss(matrix, f):

  ext = get_extended_matrix(matrix, f)  

  n = len(matrix)
  for i in range(n):

      leading = i + np.argmax(np.abs(matrix[:,i][i:]))
      ext[[i, leading]] = ext[[leading, i]] 

      ext[i] /= ext[i][i]
      row = ext[i]

      for r in ext[i + 1:]:
          r -= r[i] * row

  for i in range(n - 1, 0, -1):
      row = ext[i]
      for r in reversed(ext[:i]):
          r -= r[i] * row

  return ext[:,-1]

def J(t, y):
    jacobian = np.zeros((8, 8))
    
    jacobian[0] = [-1.71, 0.43, 8.23, 0, 0, 0, 0, 0]
    jacobian[1] = [+1.71, -8.75, 0, 0, 0, 0, 0, 0]
    jacobian[2] = [0, 0, -10.03, 0.43, 0.035, 0, 0, 0]
    jacobian[3] = [0, 8.32, 1.71, -1.12, 0, 0, 0, 0]
    jacobian[4] = [0, 0, 0, 0, -1.745, 0.43, 0.43, 0]
    jacobian[5] = [0, 0, 0, 0.69, 1.71, -280*y[7]-0.43, 0.69, -280*y[5]]
    jacobian[6] = [0, 0, 0, 0, 0, +280*y[7], -1.87, +280*y[5]]
    jacobian[7] = [0, 0, 0, 0, 0, -280*y[7], +1.87, -280*y[5]]

    return jacobian
    
def rosenbrock(y0, tau, T):
    t = np.arange(0, T, tau)
    n = int(T/tau)
    y = np.zeros((n, len(y0)))
    
    y[0] = y0
    matrix = np.eye(len(y0)) - 0.5 * tau * (1+1j) * J(t[0], y[0])
    
    for idx in range(n-1):
        right_side = f(t[idx] + tau/2, y[idx])
        y[idx+1] = y[idx] + tau * np.real(gauss(matrix, right_side))

    return t, y

#### Построение графиков

In [ ]:
t_rb, y_rb = rosenbrock(y0, 0.015, 50)
graph(t_rb, y_rb, 'Метод Розенброка tau=0.015', 'pictures/rb.1.png')

<img src='pictures/rb.1.png'>

In [ ]:
t_rb, y_rb = rosenbrock(y0, 0.01, 50)
graph(t_rb, y_rb, 'Метод Розенброка tau=0.01', 'pictures/rb.2.png')

<img src='pictures/rb.2.png'>

In [ ]:
t_rb, y_rb = rosenbrock(y0, 0.005, 50)
graph(t_rb, y_rb, 'Метод Розенброка tau=0.005', 'pictures/rb.3.png')

<img src='pictures/rb.3.png'>

## 3. Неявный метод Адамса

Будем использовать неявный метод адамса третьего порядка

$y_{n+1} = y_{n} + \tau (\frac{5}{12}f_{n+1} + \frac{8}{12}f_{n} - \frac{1}{12}f_{n-1})$

In [ ]:
class System:
    def __init__(self, t, y, idx, tau):
        
        self.t = t
        self.y = y
        self.idx = idx
        self.tau = tau

    def __call__(self, u):
        t = self.t
        y = self.y
        idx = self.idx
        val = (5/12)*f(t[idx+1], u) + (8/12)*f(t[idx], y[idx]) - (1/12)*f(t[idx-1], y[idx-1])
        return u - y[idx] - self.tau * val

def implicit_adams(y0, tau, T):
    t = np.arange(0, T, tau)
    n = int(T/tau)
    y = np.zeros((n, len(y0)))
    
    y[0] = y0
    y[1] = rungekutta2_iter(y, t, tau, 0)

    for idx in range(1, n-1):
        system = System(t, y, idx, tau)
        y[idx+1] = op.fsolve(system, y[idx])

    return t, y

#### Построение графиков

In [ ]:
t_ia, y_ia = implicit_adams(y0, 0.04, 50)
graph(t_ia, y_ia, 'Неявный метод Адамса tau=0.04', 'pictures/ia.1.png')

<img src='pictures/ia.1.png'>

In [ ]:
t_ia, y_ia = implicit_adams(y0, 0.032, 50)
graph(t_ia, y_ia, 'Неявный метод Адамса tau=0.032', 'pictures/ia.2.png')

<img src='pictures/ia.2.png'>

In [ ]:
t_ia, y_ia = implicit_adams(y0, 0.03, 50)
graph(t_ia, y_ia, 'Неявный метод Адамса tau=0.03', 'pictures/ia.3.png')

<img src='pictures/ia.3.png'>